In [ ]:
import pandas as pd
import numpy as np 
import os
from numpy import asarray
from matplotlib import image
from matplotlib import pyplot
import tensorflow 

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50

df = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv', index_col=0)
train_annotations = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train_annotations.csv', index_col=0)
images = os.listdir('../input/ranzcr-clip-catheter-line-classification/train')
seg_images = os.listdir('../input/annotated-segmented-images')

# check that length of study uids in training dataframe matches length in image bucket
assert len(set(images) - set([i+'.jpg' for i in df.index])) == 0

In [ ]:
all_labels = [
 'ETT - Abnormal',
 'ETT - Borderline',
 'ETT - Normal',
 'NGT - Abnormal',
 'NGT - Borderline',
 'NGT - Incompletely Imaged',
 'NGT - Normal',
 'CVC - Abnormal',
 'CVC - Borderline',
 'CVC - Normal',
 'Swan Ganz Catheter Present']
    

In [ ]:
df.index = [i +'.jpg' for i in df.index]
df = df.reset_index()

In [ ]:
df.head()

## Add Path As a Variable

In [ ]:
path = '../input/ranzcr-clip-catheter-line-classification/train'

In [ ]:
df['image'] = df['index'].map(lambda x: str(path+('/' + x)))

In [ ]:
df.head()

## Train test Split

In [ ]:
X = df["image"]
y = df.drop(["index", "PatientID",'image'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)
X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)

train_df = pd.concat([X_train, y_train], axis=1)
valid_df = pd.concat([X_test, y_test], axis=1)

train_df.head()

## Data Gen

In [ ]:
train_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=0.4,
        horizontal_flip=True)

train_datagen = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col = 'image',
    y_col = all_labels,
    target_size=(224, 224),
    batch_size = 16,
    seed = 42,
    class_mode = 'raw'
                  )

In [ ]:
valid_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=0.4,
        horizontal_flip=True)

valid_datagen = valid_generator.flow_from_dataframe(
    dataframe=valid_df,
    x_col = 'image',
    y_col = all_labels,
    target_size=(224, 224),
    batch_size = 16,
    seed = 42,
    class_mode = 'raw'
                  )

In [ ]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("y_test shape: " + str(y_test.shape))

In [ ]:
img_height,img_width = 224,224 
num_classes = 11
resnet=ResNet50(input_shape=[224,224,3],weights='imagenet',include_top=False)
resnet.trainable = False

In [ ]:
inputs = resnet.inputs
x = resnet(inputs)

x = tensorflow.keras.layers.GlobalAveragePooling2D()(x)
outputs = tensorflow.keras.layers.Dense(len(all_labels), activation='sigmoid')(x)
model = tensorflow.keras.Model(inputs, outputs)

model.summary()

In [ ]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_datagen,validation_data = valid_datagen,epochs = 20, verbose = 1)